In [117]:
from anise import MetaAlmanac
import requests
from datetime import datetime
import pandas as pd
from io import StringIO
import numpy as np

# Escapade 

Our escapade spacecraft were inserted into quite interesting orbits prior to their mars transfer burn, and I'd like to propagate their orbits without the help of all the nice tools we usually use in aerospace and space mission engineering (like STK and fully featured astrodynamics libraries). 

## Resources
- https://eyes.nasa.gov/apps/solar-system/#/sc_escapade_blue/distance?to=earth
- https://nyxspace.com/anise/tutorials/Tutorial%2002%20-%20Loading%20remote%20SPICE%20and%20ANISE%20files%20%28meta%20almanac%29/#metaalmanac-configuration
- https://astroutils.astronomy.osu.edu/time/bjd_explanation.html

In [152]:
# get planetary ephems
almanac = MetaAlmanac.latest()
almanac.describe(bpc=True)

=== BPC #0: `/home/zshifrel/.local/share/nyx-space/anise/earth_latest_high_prec.bpc` ===
┌─────────────────────────┬───────────────────────────────────┬───────────────────────────────────┬───────────────────────────┬────────────────────┬───────┬────────────────┐
│ Name                    │ Start epoch                       │ End epoch                         │ Duration                  │ Interpolation kind │ Frame │ Inertial frame │
├─────────────────────────┼───────────────────────────────────┼───────────────────────────────────┼───────────────────────────┼────────────────────┼───────┼────────────────┤
│ Earth PCK, ITRF93 Frame │ 2000-01-01T00:01:04.183912847 TDB │ 2002-09-26T21:30:15.692217760 TDB │ 999 days 21 h 29 min 12 s │ Chebyshev Triplet  │ 3000  │ 17             │
├─────────────────────────┼───────────────────────────────────┼───────────────────────────────────┼───────────────────────────┼────────────────────┼───────┼───────��────────┤
│ Earth PCK, ITRF93 Frame │ 2002-09-26T2

In [138]:
# Escapade spice files not yet available through https://naif.jpl.nasa.gov/pub/naif/
# so query directly from horizons
base_url = "https://ssd.jpl.nasa.gov/api/horizons.api"

# Query parameters
params = {
    'format': 'json',
    'COMMAND': "'-9'", # ESCAPADE-Blue
    'OBJ_DATA': 'YES',
    'MAKE_EPHEM': 'YES',
    'EPHEM_TYPE': 'VECTORS',
    'CENTER': '@0',  # Solar System Barycenter, for Earth Center use 399
    'START_TIME': "'2025-11-24'",
    'STOP_TIME': "'2025-11-25'",
    'STEP_SIZE': "'1 hour'",
    'VEC_TABLE': '2',
    'REF_PLANE': 'FRAME',  # Use ICRF frame
    'REF_SYSTEM': 'ICRF',
    'VEC_CORR': 'NONE',
    'OUT_UNITS': 'KM-S',  # km and km/s
    'CSV_FORMAT': 'YES'
}

resp = requests.get(base_url, params=params)
data = resp.json()

# Check if query was successful
if 'result' in data:
    print(data['result'])
else:
    print("Error or spacecraft not found:")
    print(data)

******************************************************************************
 Revised: Nov 25, 2025     EscaPADE-Blue Spacecraft / (E-S L2, Mars)        -9
                             https://escapade.ssl.berkeley.edu/

 BACKGROUND:
  ESCAPADE-Blue ("Escape and Plasma Acceleration and Dynamics Explorers") is 
  one of two twin spacecraft launched by a Blue Origin New Glenn rocket on 
  November 13, 2025 @ 20:45 UTC from Cape Canaveral LC-36 (USA).

  The low-cost (< $80 million) mission is led by UC Berkeley Space Sciences 
  Laboratory in collaboration with Rocket Labs, NASA, Advanced Space, Embry 
  Riddle University, and Blue Origin.

  It will be launched to Earth-Sun L2 and remain for a year collecting data
  on space weather. A low-altitude perigee trans-Mars injection engine burn 
  in November of 2026 will send it to Mars with the goal of characterizing 
  space weather and ionospheric variability at Mars.

  ESCAPADE will execute one major and several minor propulsive maneu

In [141]:
# parse horizons data
# format is: JDTDB (barycentric Julian date), Calendar Date, X (km), Y (km), Z (km), VX (km/s), VY (km/s), VZ (km/s)

horizons_str = data['result']

def parse_horizons_to_state(horizons_txt: str):
    state_start = '$$SOE' 
    state_end = '$$EOE'
    
    start_idx = horizons_txt.find(state_start) + len(state_start)
    end_idx = horizons_txt.find(state_end)
    data = horizons_txt[start_idx:end_idx].strip()
    
    df = pd.read_csv(
        StringIO(data),
        sep=',',
        header=None,
        comment='*',
        na_filter=False,
        usecols=[0, 1, 2, 3, 4, 5, 6, 7]
    )
    
    df.rename(columns={0: 'JDTDB', 1: 'Calendar Date', 2: 'X', 3: 'Y', 4: 'Z', 5: 'VX', 6: 'VY', 7: 'VZ'}, inplace=True)

    return df
    
states = parse_horizons_to_state(horizons_str)
states.head()

,JDTDB,Calendar Date,X,Y,Z,VX,VY,VZ
0,2.461004e+06,A.D. 2025-Nov-24 00:00:00.0000,6.872464e+07,1.185194e+08,5.130352e+07,-27.436461,12.862121,5.577624
1,2.461004e+06,A.D. 2025-Nov-24 01:00:00.0000,6.862586e+07,1.185657e+08,5.132358e+07,-27.445492,12.844177,5.569979
2,2.461004e+06,A.D. 2025-Nov-24 02:00:00.0000,6.852704e+07,1.186119e+08,5.134362e+07,-27.454514,12.826225,5.562330
3,2.461004e+06,A.D. 2025-Nov-24 03:00:00.0000,6.842818e+07,1.186580e+08,5.136363e+07,-27.463529,12.808267,5.554678
4,2.461004e+06,A.D. 2025-Nov-24 04:00:00.0000,6.832930e+07,1.187041e+08,5.138361e+07,-27.472537,12.790302,5.547021
